Use continuous geographic, hydrological, climate, and meteorological datasets as input to train neural network(NN) and random forest(RF) models, to determine the major environmentl drivers of algae growth in lakes.
Use Google Cloud Platform (GCP) to extract relative data from its Earth Engine Dataset and run further NN models

In [1]:
##connect to google earth engine
import ee
service_account = '429406912917-compute@developer.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'credentials.json')
ee.Initialize(credentials)

In [72]:
### extract climate data from ECMWF ERA 5
##details of dataset: https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY#description
## https://cds.climate.copernicus.eu/cdsapp#!/software/app-era5-explorer?tab=overview
sensor='ECMWF/ERA5/DAILY'
start_date,end_date='2020-06-01','2020-07-01'
aoi =ee.Geometry.Rectangle(-95.4, 48.88,-93.75, 49.77)
era5 = ee.ImageCollection(sensor) \
       .filterBounds(aoi)  \
       .filterDate(ee.Date(start_date), ee.Date(end_date))
era5_2mt = era5.select('mean_2m_air_temperature')
print('Number of images', era5_2mt.size().getInfo())

Number of images 30


In [76]:
##preview the average imagery
img=era5_2mt.mosaic()  #.clip(aoi)
print(img.getInfo())
from IPython.display import Image
#Image(url=img.visualize().getThumbUrl())
print("Global Temperature Jun2020 monthly avg (K)")
Image(url = img.updateMask(img.gt(0))
  .getThumbURL({'min':270, 'max': 320, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

{'type': 'Image', 'bands': [{'id': 'mean_2m_air_temperature', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}
Global Temperature Jun2020 monthly avg (K)


In [6]:
## export to imagery to Google Cloud Platform (GCP) buckets 
export_config = {
    'image': img,
    'region': aoi,
    'crs': 'EPSG:4326',
    'scale': 100000,    
    'fileFormat': 'GeoTIFF',
    'fileNamePrefix': 'ERA5_2mt',
    'bucket':   'AlgaeDrive',  
    'maxPixels':1e10,
    'description': 'ERA5 climate data download: ER5 mean_2m_air_temperature'
}
task=ee.batch.Export.image.toCloudStorage(**export_config)
#task=ee.batch.Export.image.toDrive(**export_config)
task.start()
import time
time.sleep(10)
print(task.status())

{'state': 'READY', 'description': 'ERA5 climate data download: ER5 mean_2m_air_temperature', 'creation_timestamp_ms': 1680028182464, 'update_timestamp_ms': 1680028182464, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'UQFQZPCG5XE5SKJFP2QS4TKZ', 'name': 'projects/earthengine-legacy/operations/UQFQZPCG5XE5SKJFP2QS4TKZ'}


In [ ]:
##iterate all the image to export
img_list = era5_2mt.toList(era5_2mt.size())
nImages = img_list.size().getInfo()
for i in range(nImages):
	img=ee.Image(img_list.get(i)).clip(aoi)
	product_id=img.get('PRODUCT_ID').getInfo()
	export_config = {
		'image': img.float(),
		'region': aoi_clip,
		'crs': f'EPSG:{crs_epsg}',
		'scale': 300,    #default in 300 pixels of OLCI
		'fileFormat': 'GeoTIFF',
		'fileNamePrefix': f'mali_test/{product_id}',
		'bucket':   f'mali_test',  
		'maxPixels':1e10,
		'description': 'test'
	}

In [36]:
img = era5_2mt.first().clip(aoi)
#print(img.get('bands').getInfo())
#img.get('id').getInfo()
img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'mean_2m_air_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [9, 5],
   'origin': [337, 160],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}],
 'version': 1578495016371452,
 'id': 'ECMWF/ERA5/DAILY/20160901',
 'properties': {'system:footprint': {'type': 'Polygon',
   'coordinates': [[[-95.4, 48.88],
     [-93.75, 48.88],
     [-93.75, 49.77],
     [-95.4, 49.77],
     [-95.4, 48.88]]]},
  'system:time_start': 1472688000000,
  'month': 9,
  'year': 2016,
  'system:time_end': 1472774400000,
  'system:asset_size': 36675608,
  'day': 1,
  'system:index': '20160901'}}

In [19]:
img_list = era5_2mt.toList(era5_2mt.size())
nImages = img_list.size().getInfo()
for i in range(20):
  img = ee.Image(img_list.get(i))
  #img=ee.Image(era5_2mt.get(0)).clip(aoi)
  print(img.get('PRODUCT_ID').getInfo())

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [48]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


In [49]:
# Import the Image function from the IPython.display module.
from IPython.display import Image

# Display a thumbnail of global elevation.
Image(url = dem.updateMask(dem.gt(0))
  .getThumbURL({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))